# Parameters

In [1]:
FILENAME = './datasets/churn_modelling.csv'
FEATURES_INIT = 3
FEATURES_END = 13
FEATURES_LABELENCODER = [1, 2]
FEATURES_LABELENCODER_OBJ = []
FEATURES_ONEHOT = [1]
FEATURES_ONEHOT_OBJ = []
TARGET = 13
RANDOM_STATE = 1

# Pre-processing

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV # sklearn.grid_search

# Importar el data set
dataset = pd.read_csv(FILENAME)

X = dataset.iloc[:, FEATURES_INIT:FEATURES_END].values
y = dataset.iloc[:, TARGET].values

# Codificar datos categóricos
for column in FEATURES_LABELENCODER:
    labelencoder = LabelEncoder()
    X[:, column] = labelencoder.fit_transform(X[:, column])
    FEATURES_LABELENCODER_OBJ.append(labelencoder) 


for column in FEATURES_ONEHOT:
    transformer = ColumnTransformer(
        transformers=[
            ("Tranform_{}".format(column),     # Un nombre de la transformación
             OneHotEncoder(categories='auto'), # La clase a la que transformar
             [column]                          # Las columnas a transformar.
             )
        ], remainder='passthrough'
    )

    X = transformer.fit_transform(X)
    X = X[:, 1:]
    FEATURES_ONEHOT_OBJ.append(transformer) 

    
# Dividir el data set en conjunto de entrenamiento y conjunto de testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

# Escalado de variables
scaler_x = StandardScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)


/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Model

In [15]:
# ANN Example

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


def build_classifier(optimizer, dropout, kernel_initializer, init_units):
    classifier = Sequential()
    classifier.add(Dense(units = 11, kernel_initializer = kernel_initializer,  activation = "relu", input_dim = 11))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(units = init_units, kernel_initializer = kernel_initializer,  activation = "relu"))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(units = 1, kernel_initializer = kernel_initializer,  activation = "sigmoid"))
    classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, verbose=0)
parameters = {
    'batch_size' : [32],
    'epochs' : [100], 
    'optimizer' : ['nadam'],
    'dropout': [0.1],
    'kernel_initializer': ['glorot_uniform'],
    'init_units': [11]
}

In [21]:
# RANDOMFOREST Example
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
parameters = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 3, 10],
    'random_state': [RANDOM_STATE]
}

# Training

In [22]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10,
                           n_jobs=-1)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best parameter:", best_parameters)
print("Best Accuracy:", best_accuracy)

Best parameter: {'max_depth': 10, 'n_estimators': 100, 'random_state': 1}
Best Accuracy: 0.864125
